# CRISP-DM Giai đoạn 4 & 5: Modeling & Evaluation

**Mục tiêu:**
1. Huấn luyện nhiều mô hình Machine Learning (Logistic Regression, Random Forest, SVM).
2. Đánh giá dựa trên các chỉ số: Accuracy, Precision, Recall, F1-Score, ROC-AUC.
3. So sánh và lựa chọn mô hình tốt nhất (Best Model Selection).
4. Đóng gói và lưu trữ mô hình (Deployment Preparation).

In [ ]:
import pandas as pd
import joblib
import os
import sys
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Thêm thư mục hiện tại vào path để import được src.preprocessing
sys.path.append(os.getcwd())

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.pipeline import Pipeline

# Import module tự định nghĩa
# Giả định file cấu trúc thư mục là: ./notebook.ipynb và ./src/preprocessing.py
try:
    from src.preprocessing import load_data, build_preprocessor
    print("Import module thành công!")
except ImportError as e:
    print(f"Lỗi import: {e}. Vui lòng kiểm tra lại cấu trúc thư mục src/.")

# Cấu hình đường dẫn lưu model
MODELS_DIR = os.path.join(os.getcwd(), 'models')
os.makedirs(MODELS_DIR, exist_ok=True)

## 1. Chuẩn bị Dữ liệu
Sử dụng hàm `load_data` và `build_preprocessor` đã được đóng gói để đảm bảo tính nhất quán (Reproducibility).
Dữ liệu được chia theo tỷ lệ **80% Train - 20% Test** có sử dụng `stratify` để giữ nguyên tỷ lệ nhãn.

In [ ]:
# 1. Load dữ liệu
print("Đang tải dữ liệu...")
df = load_data()

# 2. Xây dựng bộ tiền xử lý
preprocessor, X, y = build_preprocessor(df)

# 3. Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Kích thước tập Train: {X_train.shape}")
print(f"Kích thước tập Test : {X_test.shape}")

# 1. Load dữ liệu
print("Đang tải dữ liệu...")
df = load_data()

# 2. Xây dựng bộ tiền xử lý
preprocessor, X, y = build_preprocessor(df)

# 3. Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Kích thước tập Train: {X_train.shape}")
print(f"Kích thước tập Test : {X_test.shape}")

## 2. Huấn luyện đa mô hình (Multi-model Training)
Chúng ta sẽ huấn luyện 3 thuật toán phổ biến:
* **Logistic Regression:** Baseline model, dễ giải thích.
* **Random Forest:** Mô hình Ensemble, thường cho độ chính xác cao.
* **SVM:** Tốt cho không gian dữ liệu phức tạp (sử dụng `probability=True` để tính ROC-AUC).

In [ ]:
# Định nghĩa các mô hình
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

results_list = []
best_score = 0.0
best_model_name = None
best_pipeline = None

print("Bắt đầu quá trình huấn luyện...")

for name, model in models.items():
    print(f"-> Đang huấn luyện: {name}")
    
    # Tạo Pipeline: Preprocessor -> Model
    clf = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    # Fit
    clf.fit(X_train, y_train)
    
    # Predict
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)[:, 1]
    
    # Tính Metrics
    metrics = {
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred),
        "ROC-AUC": roc_auc_score(y_test, y_prob)
    }
    
    results_list.append(metrics)
    
    # Lưu best model (dựa trên F1-Score)
    if metrics['F1-Score'] > best_score:
        best_score = metrics['F1-Score']
        best_model_name = name
        best_pipeline = clf

# Chuyển kết quả sang DataFrame để dễ quan sát
results_df = pd.DataFrame(results_list).set_index("Model")
print("\nHoàn tất huấn luyện!")

## 3. So sánh và Trực quan hóa kết quả
Sắp xếp kết quả theo **F1-Score** để tìm ra mô hình cân bằng tốt nhất giữa Precision và Recall.

In [ ]:
# Hiển thị bảng kết quả
display(results_df.sort_values(by="F1-Score", ascending=False))

# Vẽ biểu đồ so sánh F1-Score
plt.figure(figsize=(10, 6))
sns.barplot(x=results_df.index, y=results_df["F1-Score"], palette="viridis")
plt.title("So sánh hiệu năng F1-Score giữa các mô hình")
plt.ylabel("F1-Score")
plt.ylim(0, 1.0)

# Hiển thị giá trị lên cột
for i, v in enumerate(results_df["F1-Score"]):
    plt.text(i, v + 0.02, f"{v:.4f}", ha='center', fontweight='bold')

plt.show()

print(f">>> Mô hình tốt nhất được chọn: {best_model_name} (F1: {best_score:.4f})")

## 4. Lưu trữ Mô hình
Lưu pipeline tốt nhất (`preprocessor` + `model`) vào file `.pkl` để có thể sử dụng lại mà không cần train lại. Đồng thời lưu log kết quả đánh giá.

In [ ]:
# 1. Lưu mô hình (.pkl)
model_path = os.path.join(MODELS_DIR, 'best_model.pkl')
joblib.dump(best_pipeline, model_path)
print(f"✅ Đã lưu mô hình tại: {model_path}")

# 2. Lưu kết quả đánh giá (.json)
json_path = os.path.join(MODELS_DIR, 'evaluation_results.json')
results_df.to_json(json_path, orient="index", indent=4)
print(f"✅ Đã lưu báo cáo tại: {json_path}")